# Undertime 

Sports betting in basketball is incredibly difficult, with bias getting in the way of placing optimal bets. It seems impossible to get it right! That's where "Undertime" comes in!

Undertime is a web app that will allow users to access to top-of-the-line ML software to help place their bets. We use a combination of ELO ratings and other advanced NBA statistics to create a Multi

## Calculating ELO Ratings

### Loading in our game data 

The dataset can be found here: https://www.kaggle.com/datasets/nathanlauga/nba-games/code?resource=download&select=teams.csv

We take data that correlates to every game from the 2003 to the current season – indicating the winner, margin of victory and other important statistics for calculating ELO rating.

In [2]:
import pandas as pd
from tqdm import tqdm
import autograd.numpy as np
import matplotlib.pyplot as plt
from autograd import grad, elementwise_grad

dat_games = pd.read_csv("games.csv",sep=",")
dat_teams = pd.read_csv("teams.csv",sep=",")

### Processing ELO Data

Foundation for the math behind calculating ELO can be found here: https://towardsdatascience.com/predicting-the-outcome-of-nba-games-with-machine-learning-a810bb768f20

ELO is calculated as follows (all teams start with 1500 ELO): 

$$
R_{i+1} = k * (S_{team} - E_{team}) + R_i
$$

where S_team is a state variable: 1 if the team wins, 0 if the team loses. E_team represents the expected win probability of the team, which is represented as:

$$
E_{team} = \frac{1}{1+10^{\frac{oppelo - teamelo}{400}}}
$$

and k is a moving constant relying on margin of vicory and difference in ELO ratings: 

$$
k = 20\frac{(MOV_{winner}+3)^{0.8}}{7.5+0.006(elo_{winner}-elo_{loser})}
$$

In [3]:
# create object where team_id -> team_abbreviation
teams = {}
ids = dat_teams['TEAM_ID']
abr = dat_teams['ABBREVIATION']
for i in range(30):
    teams[ids[i]] = abr[i]

In [6]:
# create ELO ratings where... team_id -> ELO, initialize to 1500
ELO = {}
for i in range(30): 
    ELO[abr[i]] = 1500
    
curr_season = 2003
ELO_by_season = {}
for i in range(30):
    ELO_by_season[abr[i]] = []
dat_games

for i, game in dat_games.iterrows():
    
    hteam = teams[game['HOME_TEAM_ID'] ]
    ateam = teams[game['VISITOR_TEAM_ID']] # away team
    
    # calculating S_team
    hS = game['HOME_TEAM_WINS']
    aS = 1 - hS
    
    # calculating expected value for each team
    hE = 1/(1+np.power(10, (ELO[ateam] - ELO[hteam])/400))
    aE = 1/(1+np.power(10, (ELO[hteam] - ELO[ateam])/400))
    
    # calculating k
    MOV_winner = np.absolute(game['PTS_home'] - game['PTS_away'])
    
    ELO_diff_winner = 0
    if(hS == 1):
        ELO_diff_winner = ELO[hteam] - ELO[ateam]
    else:
        ELO_diff_winner = ELO[ateam] - ELO[hteam]
        
    k = 20 * np.power(MOV_winner + 3, 0.8) / (7.5 + 0.006*ELO_diff_winner) # k = 26.92

    ELO[hteam] = k * (hS-hE)+ELO[hteam]
    ELO[ateam] = k * (aS-aE)+ELO[ateam]
    
    # if season is switching
    if curr_season != game['SEASON']:
        for i, team in enumerate(abr):
            ELO_by_season[team].append(ELO[team])
            ELO[team] = ELO[team]*0.75 + 1505*0.25
        curr_season = game['SEASON']
    
# for current season
for i, team in enumerate(abr):
    ELO_by_season[team].append(ELO[team])

In [7]:
data = []
for i, team in enumerate(ELO_by_season):
    x = [team] + ELO_by_season[team]
    data.append(x)
cols = ['Team']
for i in range(len(ELO_by_season["GSW"])):
    cols.append(2003+i)
ELO_df = pd.DataFrame(data, columns = cols)

In [8]:
import base64
import pandas as pd
from IPython.display import HTML

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(ELO_df)